In [3]:
import sys
sys.path.append('/workspace/Documents')
import os
import torch
import numpy as np 
import nibabel as nb
import pandas as pd
import Diffusion_denoising_thin_slice.denoising_diffusion_pytorch.denoising_diffusion_pytorch.conditional_diffusion as ddpm
import Diffusion_denoising_thin_slice.denoising_diffusion_pytorch.denoising_diffusion_pytorch.conditional_EDM as edm
import Diffusion_denoising_thin_slice.functions_collection as ff
import Diffusion_denoising_thin_slice.Build_lists.Build_list as Build_list
import Diffusion_denoising_thin_slice.Generator as Generator
from scipy.ndimage import gaussian_filter
from skimage.util import view_as_windows

In [4]:
patient_sheet = pd.read_excel(os.path.join('/mnt/camca_NAS/denoising/','Patient_lists', 'fixedCT_static.xlsx'),dtype={'Patient_ID': str, 'Patient_subID': str})
print('patient sheet len: ', len(patient_sheet))

patient sheet len:  100


In [8]:
i = 1
row = patient_sheet.iloc[i]
patient_id = row['Patient_ID']
patient_subid = row['Patient_subID']

image_file = os.path.join('/mnt/camca_NAS/denoising/Data/simulation',patient_id,patient_subid,'gaussian_random_0/recon.nii.gz')
image_data = nb.load(image_file).get_fdata()
print(img.shape)

(512, 512, 100)


In [9]:
img = image_data[:,:,50]
# turn into a torch tensor with shape (1, 1, x,y)
img = torch.tensor(img, dtype=torch.float32).unsqueeze(0).unsqueeze(0)
print(img.shape)

torch.Size([1, 1, 512, 512])


In [10]:
import torch
import torch.nn.functional as F
import torchvision.transforms.functional as TF
import math

# Step 1: 创建一个高斯核（2D separable）
def get_gaussian_kernel(kernel_size=21, sigma=6):
    """Create a 2D Gaussian kernel."""
    # 1D Gaussian
    x = torch.arange(kernel_size).float() - kernel_size // 2
    gauss = torch.exp(-x**2 / (2 * sigma**2))
    gauss = gauss / gauss.sum()
    # Outer product for 2D kernel
    kernel_2d = torch.outer(gauss, gauss)
    return kernel_2d

# Step 2: 定义低通滤波函数（对BCHW图像使用2D卷积）
def apply_lowpass_gaussian(img, kernel):
    """Apply Gaussian filtering to BCHW tensor using depthwise conv."""
    B, C, H, W = img.shape
    kernel = kernel.to(img.device)
    kernel = kernel.view(1, 1, *kernel.shape)  # [1,1,k,k]
    kernel = kernel.repeat(C, 1, 1, 1)  # [C,1,k,k]
    
    padding = kernel.shape[-1] // 2
    return F.conv2d(img, kernel, padding=padding, groups=C)



In [12]:
img_gauss = apply_lowpass_gaussian(img, get_gaussian_kernel(kernel_size=21, sigma=6))

In [15]:
# turn img_gauss to numpy array and save as nifti
img_gauss_np = img_gauss.squeeze().cpu().numpy()
nb.save(nb.Nifti1Image(img_gauss_np, nb.load(image_file).affine), os.path.join('/mnt/camca_NAS/denoising/Data/gauss.nii.gz'))
nb.save(nb.Nifti1Image(image_data[:,:,50], nb.load(image_file).affine), os.path.join('/mnt/camca_NAS/denoising/Data/original.nii.gz'))